# Training/Testing Set

We will split the labelled dataset into training (80%) and testing (20%) sets. These will be used for BERT/TFIDF/Doc2Vec. For BERT as a classifer, the testing set will be split in half to create validation (10%) and testing (10%) sets.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

import pickle
import time
import re
import csv
import math
import pickle
import time
import spacy
import random

2022-12-09 14:53:45.503707: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 14:53:45.728273: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-09 14:53:47.170273: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/software/standard/compiler/gcc/9.2.0/nodejs/12.14.1/lib:/apps/software/standard/core/gcc/9.2.0/lib/gcc/x86_64-pc-linux-gnu/9.2.0:/apps/software/standard/core/gcc/9.2.0/lib64:/apps/software/standard/

In [2]:
#data frame with only labeled abstracts
df_lab = pd.read_csv("/home/kno5cac/git/publicrd/data/prd/Digital_abstract_labelled/DEC-2022-labelled_abstracts.csv")

#data frame with all of the abstracts
df = pd.read_pickle("/home/kno5cac/git/publicrd/data/prd/Paper/FR_meta_and_final_tokens_23DEC21.pkl")

In [3]:
# Reset the index and create a variable index to link with PROJECT_ID
df.reset_index(inplace = True, drop = True)
df['index'] = df.index
df.head()

,PROJECT_ID,ABSTRACT,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,FY_TOTAL_COST_SUM,NUM_RECORDS,final_tokens,index
0,89996,"This is a project to explore Game-based, Metap...",Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1,1999467.0,1,project explore game base metaphor enhanced ga...,0
1,89997,Institution: Franklin Institute Science Museum...,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1,1799699.0,1,institution franklin institute science museum ...,1
2,89998,Through programs (including small group conver...,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1,1505858.0,1,program include small group conversation citiz...,2
3,89999,In partnership with the American Chemical Soci...,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1,51000.0,1,partnership american chemical society acs nati...,3
4,90001,The Center for Molecular Interfacing (CMI) wil...,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,10/1/2008,9/30/2011,"ABRUNA, HECTOR D",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008,1519821.0,NaN,1,1,1519821.0,1,center molecular interfacing cmi enable integr...,4


In [4]:
df_lab = df_lab[['PROJECT_ID','ABSTRACT','Is it related to Big-Data','label']]
df_lab

,PROJECT_ID,ABSTRACT,Is it related to Big-Data,label
0,950056,ï»¿ DESCRIPTION (provided by applicant): Adv...,1,Big-data
1,1178172,Project Summary/AbstractThe establishment of t...,1,Big-data
2,163037,DESCRIPTION (provided by applicant): The long-...,0,Non Big-data
3,880197,DESCRIPTION (provided by applicant): The most ...,0,Non Big-data
4,887331,Transsphenoidal surgery (TSS) is the best trea...,0,Non Big-data
...,...,...,...,...
1595,1290459,Extracting knowledge from data using statistic...,1,Big-data
1596,1290525,As the amount and variety of data available on...,1,Big-data
1597,1292937,"By 2030, nearly 146 million connected vehicles...",1,Big-data
1598,1293276,High-performance computing (HPC) has enabled s...,1,Big-data


In [5]:
# merge with the labelled data using PROJECT_ID.
df['PROJECT_ID'] = pd.to_numeric(df['PROJECT_ID'])
df_merge = df.merge(df_lab[['PROJECT_ID','Is it related to Big-Data','label']], how='left', on='PROJECT_ID')
len(df_merge)

1143869

In [6]:
# Save project ID of labelled data
project_id_lab = list(df_lab['PROJECT_ID'])

# Get the index of labelled abstract
subset_df = df_merge.loc[df_merge['PROJECT_ID'].isin(project_id_lab),['index', 'PROJECT_ID']]
index_lab = list(subset_df['index'])

In [7]:
# creating dataframe with just abstracts
data = df['ABSTRACT']
indicies = data.index

In [8]:
# Build the training and test sample: randomly select 80% of index_lab as training and the rest as test
random.seed(1)

# Build the index for the training and the test
index_training = random.sample(index_lab, int(0.8 * len(index_lab)))
index_test = list(set(index_lab) - set(index_training))

# sort those index (sort help to extract the Y value for each set)
index_training.sort()
index_test.sort()

In [9]:
len(index_training)

1280

In [10]:
len(index_test)

320

In [12]:
# extract project IDs based on indices
final_projectid_training = [df['PROJECT_ID'][i] for i in index_training]
final_projectid_testing = [df['PROJECT_ID'][i] for i in index_test]

In [13]:
final_projectid_training = pd.DataFrame(final_projectid_training)
final_projectid_testing = pd.DataFrame(final_projectid_testing)

In [14]:
# save project IDs
final_projectid_training.to_csv('final_projectid_training.csv', index=False) 
final_projectid_testing.to_csv('final_projectid_testing.csv', index=False)

## Split training set into two (training and validation) for BERT as a classifier

In [15]:
# extract project IDs based on indices
final_projectid_bert = [df['PROJECT_ID'][i] for i in index_test]

# Build the validation and training sample: randomly select 50% of testing set as validation and rest as testing
random.seed(2)

# Build the index for the training and the test
final_projectid_bert_testing = random.sample(final_projectid_bert, int(0.5 * len(final_projectid_bert)))
final_projectid_bert_validation = list(set(final_projectid_bert) - set(final_projectid_bert_testing))

In [16]:
len(final_projectid_bert)

320

In [17]:
len(final_projectid_bert_testing)

160

In [18]:
len(final_projectid_bert_validation)

160

In [19]:
final_projectid_bert_testing = pd.DataFrame(final_projectid_bert_testing)
final_projectid_bert_validation = pd.DataFrame(final_projectid_bert_validation)

In [20]:
final_projectid_bert_testing.to_csv('final_projectid_bert_testing.csv', index=False) 
final_projectid_bert_validation.to_csv('final_projectid_bert_validation.csv', index=False)